In [9]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest
import Plot_Hist

# Plotting and helper functions

In [3]:
def build_sn_legend_handles(sn_keys, palette_dict, linestyles_dict, linewidths_dict=None):
    return_handles = []
    for sn in sn_keys:
        color = palette_dict[sn]
        linestyle = linestyles_dict[sn]
        linewidth = linewidths_dict[sn] if linewidths_dict else 1
        line2d = Line2D([0], [0], label=sn, marker=None, 
                        color=color, linewidth=linewidth)
        return_handles.append(line2d)
    return return_handles

In [ ]:
def plot_annual_kwh_hist(
    ax, 
    df, 
    min_max_and_bin_size, 
    x_col='annual_kwh', 
    include_over_underflow=False, 
    stat='count', 
    plot_sns=False, 
    hist_plot_kwargs=None, 
    keep_edges_opaque=True, 
    div_drawn_width_by=None, 
    relative_position_idx=None, 
    run_set_general_plotting_args=True, 
    orient='v', 
    **kwargs
):
    return Plot_Hist.plot_hist(
        ax=ax, 
        df=df, 
        x_col=x_col, 
        min_max_and_bin_size=min_max_and_bin_size, 
        include_over_underflow=include_over_underflow, 
        stat=stat, 
        plot_sns=plot_sns, 
        hist_plot_kwargs=hist_plot_kwargs, 
        keep_edges_opaque=keep_edges_opaque, 
        div_drawn_width_by=div_drawn_width_by, 
        relative_position_idx=relative_position_idx, 
        run_set_general_plotting_args=run_set_general_plotting_args, 
        orient=orient, 
        **kwargs
    )

def plot_multiple_annual_kwh_hist(
    ax, 
    dfs_w_args, 
    min_max_and_bin_size, 
    x_col='annual_kwh', 
    include_over_underflow=False, 
    stat='count', 
    plot_sns=False, 
    hist_plot_kwargs_list=None, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=False, 
    draw_single_idx_full_width=None, 
    orient='v', 
    **kwargs
):
    return Plot_Hist.plot_multiple_hists(
        ax=ax, 
        dfs_w_args=dfs_w_args, 
        x_col=x_col, 
        min_max_and_bin_size=min_max_and_bin_size, 
        include_over_underflow=include_over_underflow, 
        stat=stat, 
        plot_sns=plot_sns, 
        keep_edges_opaque=keep_edges_opaque, 
        include_hatches=include_hatches, 
        draw_side_by_side=draw_side_by_side, 
        draw_single_idx_full_width=draw_single_idx_full_width, 
        orient=orient, 
        **kwargs
    )

In [ ]:
#NOW IN PLOT_GENERAL
def save_fig(fig, save_dir, save_name):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, save_name)
    fig.savefig(save_path)

#NOW IN PLOT_GENERAL
def get_subplots_adjust_args(left=None, right=None, bottom=None, top=None, wspace=None, hspace=None):
    # Values are set to default values (rcParams["figure.subplot.[name]"]) if not specified
    # wspace: The width of the padding between subplots, as a fraction of the average Axes width.
    # hspace: The height of the padding between subplots, as a fraction of the average Axes height.
    # left, right, bottom, top are self-explanatory
    left   = mpl.rcParams["figure.subplot.left"]   if left   is None else left
    right  = mpl.rcParams["figure.subplot.right"]  if right  is None else right
    bottom = mpl.rcParams["figure.subplot.bottom"] if bottom is None else bottom
    top    = mpl.rcParams["figure.subplot.top"]    if top    is None else top
    wspace = mpl.rcParams["figure.subplot.wspace"] if wspace is None else wspace
    hspace = mpl.rcParams["figure.subplot.hspace"] if hspace is None else hspace
    return {'left'   : left, 
            'right'  : right, 
            'bottom' : bottom, 
            'top'    : top, 
            'wspace' : wspace, 
            'hspace' : hspace}

#NOW IN PLOT_GENERAL AS get_subplots_adjust_args_std_3x1
def get_subplots_adjust_args_annual_kwh_3x1():
    top = 0.95
    bottom = 0.075
    left = 0.075
    right = 0.975
    hspace=0.5
    #------------------
    subplots_adjust_args = get_subplots_adjust_args(top=top, bottom=bottom, left=left, right=right, hspace=hspace)
    return subplots_adjust_args

#NOW IN PLOT_GENERAL AS get_subplots_adjust_args_std_2x1
def get_subplots_adjust_args_annual_kwh_2x1():
    top = 0.95
    bottom = 0.075
    left = 0.075
    right = 0.975
    hspace=0.35
    #------------------
    subplots_adjust_args = get_subplots_adjust_args(top=top, bottom=bottom, left=left, right=right, hspace=hspace)
    return subplots_adjust_args

# NOT MOVING YET BECAUSE USED LIKE 100 TIMES, AND I DON"T FEEL LIKE ADDING DOVSOutage. to each
def get_mjr_mnr_nms_from_cds(mjr_mnr_cause_cds, df_outg_cause_dim, combine_output=False):
    assert(len(mjr_mnr_cause_cds)==2)
    if df_outg_cause_dim is None:
        conn_outages = Utilities.get_utldb01p_oracle_connection()
        df_outg_cause_dim = pd.read_sql('SELECT * FROM DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM', conn_outages)        
    sub_df = df_outg_cause_dim[(df_outg_cause_dim['MJR_CAUSE_CD']==mjr_mnr_cause_cds[0]) & 
                               (df_outg_cause_dim['MNR_CAUSE_CD']==mjr_mnr_cause_cds[1])]
    mjr_cause_nm = sub_df.iloc[0]['MJR_CAUSE_NM']
    mnr_cause_nm = sub_df.iloc[0]['MNR_CAUSE_NM']
    mjr_mnr_cause_nms = (mjr_cause_nm, mnr_cause_nm)
    if combine_output:
        mjr_mnr_cause_nms = '-'.join(mjr_mnr_cause_nms)
    return mjr_mnr_cause_nms

In [ ]:
def get_color_patch(color, marker='o', 
                    markersize=7.5, alpha=1.0, label='', 
                    **kwargs):
    markerfacecolor = kwargs.get('markerfacecolor', color)
    patch = Line2D([0], [0], marker=marker, color=color, markerfacecolor=markerfacecolor, 
                   markersize=markersize, alpha=alpha, linewidth=0, 
                   label=label)
    return patch